# Prêt à dépenser : Construire un modèle de scoring


## Contexte

"Prêt à dépenser" (Home Credit) est une société financière qui propose des crédits à la consommation pour des personnes ayant peu ou pas d'historique de prêt.
Pour accorder un crédit à la consommation, l'entreprise calcule la probabilité qu'un client le rembourse, ou non. Elle souhaite donc développer un algorithme de scoring pour aider à décider si un prêt peut être accordé à un client.

Les chargés de relation client seront les utilisateurs du modèle de scoring. Puisqu'ils s'adressent aux clients, ils ont besoin que votre modèle soit facilement interprétable. Les chargés de relation souhaitent, en plus, disposer d'une mesure de l'importance des variables qui ont poussé le modèle à donner cette probabilité à un client.


## Chargement des modules du projet

Afin de simplifier le Notebook, le code métier du projet est placé dans le répertoire [src/](../src/).


In [ ]:
# Import project modules from source directory

# system modules
import os
import sys

# Append source directory to system path
src_path = os.path.abspath(os.path.join("../src"))
if src_path not in sys.path:
    sys.path.append(src_path)

# helper functions
import data.helpers as data_helpers
import features.helpers as feat_helpers
import visualization.helpers as vis_helpers


Nous allons utiliser le langage [Python](https://www.python.org/about/gettingstarted/), et présenter ici le code, les résultats et l'analyse sous forme de [Notebook JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html).

Nous allons aussi utiliser les bibliothèques usuelles d'exploration et analyse de données, afin d'améliorer la simplicité et la performance de notre code :
  * [NumPy](https://numpy.org/doc/stable/user/quickstart.html) et [Pandas](https://pandas.pydata.org/docs/user_guide/index.html) : effectuer des calculs scientifiques (statistiques, algèbre, ...) et manipuler des séries et tableaux de données volumineuses et complexes
  * [scikit-learn](https://scikit-learn.org/stable/getting_started.html) et [XGBoost](https://xgboost.readthedocs.io/en/latest/get_started.html) : pour effectuer des analyses prédictives 
  * [Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html), [Pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html), [Seaborn](https://seaborn.pydata.org/tutorial/function_overview.html) et [Plotly](https://plotly.com/python/getting-started/) : générer des graphiques lisibles, intéractifs et pertinents


In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# sklearn preprocessing for dealing with categorical variables
from sklearn.model_selection import train_test_split
import xgboost as xgb

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Prevent excessive memory usage used by plotly
DRAW_PLOTS: bool = True


## Chargement des données

Les données mises à disposition sont issues de [Home Credit](https://www.homecredit.net/) et plus précisément de la compétition hébergée sur Kaggle [Home Credit Default Risk - Can you predict how capable each applicant is of repaying a loan?](https://www.kaggle.com/c/home-credit-default-risk)

Les données sont fournies sous la forme de plusieurs fichiers CSV pouvant être liés entre eux de la manière suivante :

![Home Credit data relations](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

_source : [Introduction: Home Credit Default Risk Competition](https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction) by [Will Koehrsen](https://www.kaggle.com/willkoehrsen)_


In [ ]:
# Download and extract the raw data
data_helpers.download_extract_zip(
    zip_file_url="https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip",
    files_names=(
        "application_test.csv",
        "application_train.csv",
        "bureau_balance.csv",
        "bureau.csv",
        "credit_card_balance.csv",
        "installments_payments.csv",
        "POS_CASH_balance.csv",
        "previous_application.csv",
    ),
    target_path="../data/raw/",
)


Nous allons charger toutes les données des fichiers `application_{train|test}.csv` dans le même DataFrame, afin de travailler les données en commun des jeux d'entraînement (variable `TARGET` vaut `O` : le client n'a pas fait défaut ou `1` : le client a fait défaut) et de test(variable `TARGET` non définie). Nous les séparerons à nouveau au moment de l'entrainement et évaluation de nos modèles.

Les fichiers contiennent un grand nombre de variables booléennes et categorielles que nous pouvons déjà typer comme telles.


In [ ]:
# Read column names
application_train_column_names = pd.read_csv(
    "../data/raw/application_train.csv", nrows=0
).columns.values
application_test_column_names = pd.read_csv(
    "../data/raw/application_test.csv", nrows=0
).columns.values

# TARGET variable must be present in the Train datase
if "TARGET" not in application_train_column_names:
    raise ValueError(
        "TARGET column not found in application_train.csv. Please check that the file is not corrupted."
    )

# SK_ID_CURR variable must be present in the Train datase
if "SK_ID_CURR" not in application_train_column_names:
    raise ValueError(
        "SK_ID_CURR column not found in application_train_column_names.csv. Please check that the file is not corrupted."
    )

# Train and Test datasets must have the same variables, except for the TARGET variable
if list(
    application_train_column_names[application_train_column_names != "TARGET"]
) != list(application_test_column_names):
    raise ValueError(
        "Column names in application_train.csv and application_test.csv do not match. Please check that the files are not corrupted."
    )

# Set column types according to fields description (../data/raw/HomeCredit_columns_description.csv)
# Categorical variables
column_types = {
    col: "category"
    for col in application_train_column_names
    if col.startswith(("NAME_",))
    or col.endswith(("_TYPE"))
    or col
    in [
        "CODE_GENDER",
        "WEEKDAY_APPR_PROCESS_START",
        "FONDKAPREMONT_MODE",
        "HOUSETYPE_MODE",
        "WALLSMATERIAL_MODE",
        "EMERGENCYSTATE_MODE",
    ]
}
# Boolean variables
column_types |= {
    col: bool
    for col in application_train_column_names
    if col.startswith(("FLAG_", "REG_", "LIVE_"))
}

# Load application data
app_train_df = pd.read_csv(
    "../data/raw/application_train.csv",
    dtype=column_types,
    true_values=["Y", "Yes", "1"],
    false_values=["N", "No", "0"],
    na_values=["XNA"],
)
app_test_df = pd.read_csv(
    "../data/raw/application_test.csv",
    dtype=column_types,
    true_values=["Y", "Yes", "1"],
    false_values=["N", "No", "0"],
    na_values=["XNA"], # bad values
)
app_test_df['TARGET'] = -1 # identify test data

# Merge Train and Test datasets
app_df = app_train_df.append(app_test_df)

# Let's display basic statistical info about the data
app_df.describe(include="all")


In [ ]:
app_df.info()

Le jeu de données contient 122 variables, dont la variable cible que nous devons estimer : `TARGET`. Parmis ces variables, nous avons :
- 34 variables booléennes
- 14 variables catégorielles
- 74 variables numériques

## Analyse exploratoire

Nous allons analyser la distribution de quelques variables.

### Variable cible

Voyons spécifiquement la distribution de la variable `TARGET` qui est celle que nous devrons estimer par la suite.
Les valeurs nulles représentent notre jeu d'entrainement.
Nous pouvons oberver que nous avons à faire à un problème de __classification binaire déséquilibré__ (il y a deux valeurs possibles, mais les deux valeurs ne sont pas également représentées).
Ceci va influencer la manière dont nous allons construire et entraîner notre modèle.


In [ ]:
# Let's plot the distribution of the TARGET variable
if DRAW_PLOTS:
    px.bar(
        app_df["TARGET"].replace({
            "0": "0 : payments OK", 
            "1": "1 : payment difficulties", 
            "-1": "-1 : undefined (test dataset)",
        }).value_counts(),
        title="Distribution of TARGET variable",
        width=800,
        height=400,
    ).update_xaxes(
        title="TARGET",
    ).update_yaxes(title="Count")


### Valeurs vides

Nous voyons que toutes les variables ont moins de 30% de valeurs vides, et près de la moitié a moins de 1% de valeurs vides. Le jeu de données est donc relativement bien rempli, ce qui ne devrait pas poser de problème pour la suite.


In [ ]:
# Let's display variables with missing values ratio
if DRAW_PLOTS:
    vis_helpers.plot_empty_values(app_df)


### Valeurs impossibles

Quelques valeurs présentes dans les données semblent impossibles. Nous allons supprimer ces "outliers".


In [ ]:
# Define data constraints
data_constraints = {
    "DAYS_EMPLOYED": {"min": -35000, "max": 0,}, # max 100 years, only negative values
}

if DRAW_PLOTS:
    # Let's display box plots for variables with outliers
    vis_helpers.plot_boxes(app_df, plot_columns=data_constraints.keys(), categorical_column="TARGET")

# Remove values that are outside possible range
app_df = feat_helpers.drop_impossible_values(
    app_df, constraints=data_constraints,
)


### Variables quantitatives

Nous allons simplement afficher la distribution de quelques variables numériques. Nous voyons déjà que selon la valeur de TARGET, la distribution (moyenne) des variables peut être sensiblement différente.


In [ ]:
# Draw the BoxPlots of some numeric columns, split per Target
if DRAW_PLOTS:
    vis_helpers.plot_boxes(app_df,
        plot_columns=[
            "AMT_INCOME_TOTAL",
            "AMT_CREDIT",
            "AMT_ANNUITY",
            "AMT_GOODS_PRICE",
            "DAYS_BIRTH",
            "DAYS_EMPLOYED",
            "OWN_CAR_AGE",
            "REGION_RATING_CLIENT",
            "REGION_RATING_CLIENT_W_CITY",
            "EXT_SOURCE_1",
            "EXT_SOURCE_2",
            "EXT_SOURCE_3",
            "DAYS_LAST_PHONE_CHANGE",
            "AMT_REQ_CREDIT_BUREAU_YEAR",
        ],
        categorical_column="TARGET",
    )


### Variables qualitatives

De la même manière, nous allons simplement afficher la distribution de quelques variables catégorielles. Nous voyons déjà que selon la valeur de TARGET, la distribution (répartition entre classes) des variables peut être sensiblement différente (`TARGET=0` pour 77,6% des `NAME_CONTRACT_TYPE="Cash loans"`, tandis que `TARGET=0` pour 93,1% des `NAME_CONTRACT_TYPE="Revolving loans"`).
Certaines variables ont une répartition très inégale entre classes (`FLAG_MOBIL` vaut systématiquement `True` et jamais `False`).


In [ ]:
# Draw the Bar charts of some categorical columns, split per Target
if DRAW_PLOTS:
    vis_helpers.plot_categories_bars(app_df,
        plot_columns=[
            "NAME_CONTRACT_TYPE",
            "CODE_GENDER",
            "FLAG_OWN_CAR",
            "FLAG_OWN_REALTY",
            "NAME_INCOME_TYPE",
            "NAME_EDUCATION_TYPE",
            "NAME_FAMILY_STATUS",
            "NAME_HOUSING_TYPE",
            "OCCUPATION_TYPE",
            "FLAG_MOBIL",
        ],
        categorical_column="TARGET",
    )


In [ ]:
X = app_df.loc[app_df["TARGET"] >= 0].drop(["TARGET"], axis=1)
y = app_df.loc[app_df["TARGET"] >= 0, "TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Feature Engineering

Afin d'apporter plus de sens aux données que nous allons fournir à nos modèles, nous pouvons faire appel aux experts métier qui peuvent nous indiquer des informations qui sont réputées importantes afin de prédire si un client risque d'avoir des problèmes de remboursement ou non.

Les informations métier pertinentes sont :
- Montant emprunté / Prix du bien acheté : `AMT_CREDIT / AMT_GOODS_PRICE`
- Montant des annuités / Montant emprunté : `AMT_ANNUITY / AMT_CREDIT`
- Montant des annuités / Revenu annuel : `AMT_ANNUITY / AMT_INCOME_TOTAL`
- Ancienneté au travail / Age : `DAYS_EMPLOYED / DAYS_BIRTH`


In [ ]:
# Create the new features
feat_app_df = app_df.copy()
feat_app_df["CREDIT_PRICE_RATIO"]=feat_app_df["AMT_CREDIT"]/feat_app_df["AMT_GOODS_PRICE"]
feat_app_df["ANNUITY_CREDIT_RATIO"]=feat_app_df["AMT_ANNUITY"]/feat_app_df["AMT_CREDIT"]
feat_app_df["ANNUITY_INCOME_RATIO"]=feat_app_df["AMT_ANNUITY"]/feat_app_df["AMT_INCOME_TOTAL"]
feat_app_df["EMPLOYED_BIRTH_RATIO"]=feat_app_df["DAYS_EMPLOYED"]/feat_app_df["DAYS_BIRTH"]

# Draw the BoxPlots for these features
if DRAW_PLOTS:
    vis_helpers.plot_boxes(feat_app_df,
        plot_columns=[
            "CREDIT_PRICE_RATIO",
            "ANNUITY_CREDIT_RATIO",
            "ANNUITY_INCOME_RATIO",
            "EMPLOYED_BIRTH_RATIO",
        ],
        categorical_column="TARGET",
    )


In [ ]:
X_feat = feat_app_df.loc[feat_app_df["TARGET"] >= 0].drop(["TARGET"], axis=1)
X_feat_train, X_feat_test, y_feat_train, y_feat_test = train_test_split(X_feat, y, test_size=0.2, random_state=42)


## Préparation des données

Afin que nos modèles puissent exploiter au mieux les données, nous allons les transformer.


### Encodage des catégories

Lorsque les données qualitatives ne sont pas ordinales (on ne peu pas les classer selon un certain ordre), l'encodage "One Hot Encoding" sera plus performant que le "Label Encoding".


In [ ]:
# Encode categorical variables with One Hot Encoding
encoded_app_df = pd.get_dummies(feat_app_df, dtype=bool)

encoded_app_df.describe(include="all")

In [ ]:
X_encoded = encoded_app_df.loc[encoded_app_df["TARGET"] >= 0].drop(["TARGET"], axis=1)
X_encoded_train, X_encoded_test, y_encoded_train, y_encoded_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


Nous avons ici créé 133 nouvelles variables booléennes qui correspondent aux différentes classes de chacune des 14 anciennes variables catégorielles qui ont été encodées et supprimées.


### Normalisation des données

Afin d'éviter que certains modèles pondèrent l'importance de certaines variables à cause de leur ordre de grandeur, nous allons normaliser chaque variable afin de les ramener à une moyenne nulle et une variance de 1.


In [17]:
# Scale each variable of the DataFrame
from sklearn.preprocessing import StandardScaler


# define scaler
scaler = StandardScaler()
# fit scaler on train data only, to avoid data leak
scaler.fit(X_encoded_train)
# transform the dataset
scaled_app_df = pd.DataFrame(
    scaler.transform(X_encoded), columns=X_encoded.columns, index=X_encoded.index
)

scaled_app_df["TARGET"] = y
scaled_app_df.describe(include="all")


,SK_ID_CURR,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
count,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252125.000000,251881.000000,252137.000000,252137.000000,...,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000,252137.000000
mean,0.001472,0.000350,-0.003197,-0.000650,-0.000245,-0.000268,-0.000399,-0.000189,0.001206,0.000149,...,-0.000676,-0.001246,0.000243,-0.000122,0.002029,-0.001475,-0.000317,-0.000204,-0.000520,0.086600
std,0.999864,1.000093,1.001230,0.999214,0.909802,1.000003,1.000471,0.999706,1.002669,1.000114,...,0.998135,0.992970,1.001556,0.999178,1.001409,0.998964,0.998842,1.000011,0.997082,0.281248
min,-1.730642,-0.773443,-1.457802,-0.653360,-0.528387,-1.393533,-1.764798,-1.363165,-1.487858,-2.848137,...,-0.175574,-0.088228,-0.077782,-0.073676,-0.522845,-0.519530,-0.134077,-1.042947,-0.088456,0.000000
25%,-0.864812,-0.773443,-1.457802,-0.653360,-0.223130,-0.820459,-0.733916,-0.808767,-0.785686,-0.762753,...,-0.175574,-0.088228,-0.077782,-0.073676,-0.522845,-0.519530,-0.134077,-1.042947,-0.088456,0.000000
50%,0.000980,-0.773443,0.685964,-0.653360,-0.064966,-0.221947,-0.135488,-0.266422,-0.146533,0.053706,...,-0.175574,-0.088228,-0.077782,-0.073676,-0.522845,-0.519530,-0.134077,0.958821,-0.088456,0.000000
75%,0.867471,1.292920,0.685964,0.655950,0.124831,0.535554,0.532710,0.384392,0.562649,0.817737,...,-0.175574,-0.088228,-0.077782,-0.073676,-0.522845,-0.519530,-0.134077,0.958821,-0.088456,0.000000
max,1.733855,1.292920,0.685964,24.223535,410.608725,8.458235,15.723622,9.375275,3.731312,1.988086,...,5.695597,11.334312,12.856491,13.572916,1.912612,1.924815,7.458419,0.958821,11.305098,1.000000


In [18]:
X_scaled = scaled_app_df.loc[scaled_app_df["TARGET"] >= 0].drop(["TARGET"], axis=1)
X_scaled_train, X_scaled_test, y_scaled_train, y_scaled_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


Nous avons ici équilibré les ordres de grandeur de chaque variables, afin que nos fututs modèles ne soient pas influencés par leur différence.


### Imputation des valeurs manquantes

Afin d'éviter que certains modèles ne puissent être utilisés à cause des valeurs manquantes, nous allons remplacer toutes les valeurs nulles par leur meilleure estimation possible.


In [19]:
# Impute missing values by modeling each feature with missing values as a function of other features in a round-robin fashion
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


# define imputer
imputer = IterativeImputer(n_nearest_features=10, verbose=2)
# fit scaler on train data only, to avoid data leak
imputer.fit(X_scaled_train)
# transform the dataset
imputed_app_df = pd.DataFrame(
    imputer.transform(X_scaled), columns=X_scaled.columns, index=X_scaled.index,
)

imputed_app_df["TARGET"] = y
imputed_app_df.describe(include="all")

In [ ]:
X_imputed = imputed_app_df.loc[imputed_app_df["TARGET"] >= 0].drop(["TARGET"], axis=1)
X_imputed_train, X_imputed_test, y_imputed_train, y_imputed_test = train_test_split(
    X_imputed, y, test_size=0.2, random_state=42
)



---

# Annexe

Les Notebooks Kaggle [Introduction: Home Credit Default Risk Competition](https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction) (et suivants) de [Will Koehrsen](https://www.kaggle.com/willkoehrsen) ont été d'une très grande aide dans l'exploration des données.
